In [73]:
import pandas as pd

In [74]:
OUT_DIR = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/"

### Voting system

###### Import centrifuge data

In [106]:
def parseContigName(name):
    name = name.split("_")
    return name[1]+"_contig"+name[0]

In [115]:
centrifuge = pd.read_csv(OUT_DIR+"contigsVsVIRALtax.centrifuge.csv2",sep='\t',header=None)
header = ["C","LINEAGE"]
centrifuge.columns = header
centrifuge["CONTIG"] = centrifuge["C"].apply(parseContigName)
centrifuge.set_index("CONTIG",inplace=True)
centrifuge.head()

,C,LINEAGE
CONTIG,,
L2_contig99,99_L2_8Lpp_out_L2356,"root;Viruses;dsDNA viruses, no RNA stage;Caudo..."
X1_contig9997,9997_X1_8Lpp_out_L531,"root;Viruses;dsDNA viruses, no RNA stage;Caudo..."
K1_contig9996,9996_K1_8Lpp_out_L526,"root;Viruses;dsDNA viruses, no RNA stage;Caudo..."
T2_contig998,998_T2_8Lpp_out_L903,root;Viruses;unclassified bacterial viruses;en...
M1_contig9977,9977_M1_8Lpp_out_L1445,root;Viruses;unclassified bacterial viruses;en...


###### Import LCA data

In [116]:
lca = pd.read_csv(OUT_DIR+"contigGenesVsVIRALtax.MeganMapping.csv",
                                sep=';',header=None)
header = ["PROTEIN","LINEAGE"]
lca.columns = header

getContig = lambda x : x.split(">")[1]
lca["CONTIG"] = lca["PROTEIN"].apply(getContig)
lca.set_index("CONTIG",inplace=True)
lca.head()

,PROTEIN,LINEAGE
CONTIG,,
Ab1_contig1,gene_12|GeneMark.hmm|261_aa|-|5686|6471_>Ab1_c...,"root;Viruses;dsDNA viruses, no RNA stage;"
Ab1_contig2,gene_36|GeneMark.hmm|143_aa|+|3025|3456_>Ab1_c...,"root;Viruses;dsDNA viruses, no RNA stage;"
Ab1_contig392,gene_726|GeneMark.hmm|494_aa|-|2|1483_>Ab1_con...,"root;Viruses;dsDNA viruses, no RNA stage;"
Ab1_contig768,gene_1155|GeneMark.hmm|181_aa|-|14319|14864_>A...,"root;Viruses;dsDNA viruses, no RNA stage;"
Ab1_contig1084,gene_1355|GeneMark.hmm|79_aa|+|3473|3712_>Ab1_...,"root;Viruses;dsDNA viruses, no RNA stage;"


###### Generate the trees

In [148]:
#Define the tree class 
class Tree(object):
    def __init__(self):
        self.child = []
        self.name = None
        self.times = None
    def __init__(self,name):
        self.name = name
        self.times = 1
        self.child = []
    def addChild(self, name):
        found = False
        for c in self.child:
            if c.name == name:
                c.times += 1
                found = True
        if found == False:
            self.child.append(Tree(name))
    def insertNode(self,father,child):
        if self.name == father:
            self.addChild(child)
            return True
        else:
            for c in self.child:
                c.insertNode(father,child)
    def insertRoot(self,node):
        if self.name == node:
            self.times += 1
    def printTree(self):
        print self.name, self.times
        for c in self.child:
            c.printTree()
    def equalChilds(self):
        if len(self.child) > 1:
            first = self.child[0].times
            for i in range(1,len(self.child)):
                if self.child[i].times != first:
                    return False
            return True
        else:
            return False
    def findPath(self,path):
        path.append((self.name,self.times))
        maxChild = 0
        nextChild = None
        if ~(self.equalChilds()):
            for c in self.child:
                if c.times > maxChild:
                    maxChild = c.times
                    nextChild = c
        if nextChild != None:
            nextChild.findPath(path)
        return path

In [149]:
#Fill the trees with centrifuge data and then with lca data (actually the order does not matters)
#The thing is that the LCA annotations have a ';' at the end, generating an extra level that must be removed
trees = {}
for index,row in lca.iterrows():
    l = row["LINEAGE"].split(';')[:-1]#Here is when the extra level is removed
    if index in trees:
        t = trees[index]
        t.insertRoot(l[0])
        for i in range(1,len(l)):
                t.insertNode(l[i-1],l[i])
        trees[index] = t
    else:
        t = Tree(l[0])
        for i in range(1,len(l)):
                t.insertNode(l[i-1],l[i])
        trees[index] = t
print "LCA annotations inserted"

for index,row in centrifuge.iterrows():
    l = row["LINEAGE"].split(';')
    if index in trees:
        t = trees[index]
        t.insertRoot(l[0])
        for i in range(1,len(l)):
                t.insertNode(l[i-1],l[i])
        trees[index] = t
    else:
        t = Tree(l[0])
        for i in range(1,len(l)):
                t.insertNode(l[i-1],l[i])
        trees[index] = t
print "CENTRIFUGE annotations inserted"

LCA annotations inserted
CENTRIFUGE annotations inserted


In [152]:
fw = open(OUT_DIR+"contigsVsVIRALtax.votingSystem.txt",'w')
for contig in trees:
    lineage = trees[contig].findPath([])
    lineage = [x[0] for x in lineage]
    lineage = "; ".join(lineage)
    print >> fw, contig + '\t' + lineage
fw.close()

In [153]:
#Edit the file in excel to add the taxonomy and propperly put all the hierarchical annotation.